In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from config import g_key
gmaps.configure(api_key=g_key)

In [4]:
weather_df = pd.read_csv("output_data/weather.csv")
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Carnarvon,0,AU,1595085672,66,-24.87,113.63,55.40,12.75
1,George Town,40,MY,1595085613,94,5.41,100.34,78.80,4.70
2,Hilo,90,US,1595085673,88,19.73,-155.09,72.00,5.82
3,Petukhovo,80,RU,1595085673,58,55.07,67.90,81.54,3.27
4,Hobart,24,AU,1595085447,66,-42.88,147.33,50.00,13.87


In [10]:
humidity = weather_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = weather_df[["Lat", "Lng"]]

In [16]:

# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=1.5)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:

city_df = weather_df.loc[(weather_df["Max Temp"] > 80) & (weather_df["Max Temp"] < 90) & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0), :]
city_df = city_df.dropna(how='any')
city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
40,Ternate,0,ID,1595085507,76,0.80,127.40,81.00,2.89
63,Albany,0,US,1595085707,62,42.60,-73.97,81.00,1.16
91,Rio Branco,0,BR,1595085720,55,-9.97,-67.81,86.00,3.36
147,Chabahar,0,IR,1595085753,74,25.29,60.64,89.10,2.10
154,Lucapa,0,AO,1595085757,18,-8.42,20.74,82.56,6.02


In [20]:
# Create a hotel_df
hotel_df = city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

In [22]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [24]:
for index, row in hotel_df.iterrows():
    # get data
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"First found hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    
# Print done once searching is completed
print("Done")

First found hotel in Ternate is MUARA HOTEL and MALL TERNATE.
------------
Missing field/result... skipping.
------------
First found hotel in Rio Branco is Amazônia Palace Hotel.
------------
First found hotel in Chabahar is Laleh Hotel.
------------
Missing field/result... skipping.
------------
First found hotel in Lázaro Cárdenas is Hotel de Casablanca.
------------
First found hotel in Chaman is Sakhy Hotal Wah Mosafir Khana.
------------
First found hotel in Kiryat Gat is Desert Gat.
------------
First found hotel in Avila is Hotel Restaurante El Pentagono.
------------
First found hotel in Uvat is Gostinitsa Biatlon.
------------
First found hotel in Almuñécar is Hotel Helios Costa Tropical.
------------
Done


In [27]:
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
40,Ternate,ID,0.80,127.40,MUARA HOTEL and MALL TERNATE
63,Albany,US,42.60,-73.97,
91,Rio Branco,BR,-9.97,-67.81,Amazônia Palace Hotel
147,Chabahar,IR,25.29,60.64,Laleh Hotel
154,Lucapa,AO,-8.42,20.74,


In [35]:
marker_locations = hotel_df[["Lat","Lng"]]

infobox = hotel_df["City"]

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))